In [26]:
from math import sqrt
from queue import PriorityQueue

'''the class Node takes as input:
1) n: the name of the node or city
2) g: the euclidean distance of the node from node A
3) h: the heuristic value of the node
4) f: g+h
5) path_node: the path of reaching this node from A

The methods of this class are:
1) path_list: gives the path of the path from A to the current node
2) __lt__: The priority queue does not work when it has 2 nodes having the same f value. 
So this function specifies that we remove the first node encountered'''
class Node:
    def __init__(self, n, g, h, f, path_node):
        self.n = n
        self.g = g
        self.h = h
        self.f = f
        self.path_node = path_node
    def path_list(self, n, path_node):
        self.path_node.append(n)
        return self.path_node
    def __lt__(self, other):
        return self.f < other.f

        
'''readFile() takes as input:
1) file_name: the file containing the information about the cities as input 

The function returns a dictionary, where the city node is the key and the coordinates are its value '''    
def readFile(file_name):
    f = open(file_name, "r")
    print('The number of cities are : ' +f.readline())
    dict = {}
    for line in f:
        x = line.split()
        a = x[0]
        b = [x[1], x[2]]
        dict[a] = b
    print("The cities and their coordinates are:",dict)
    return dict        
        

'''The cost(dict) function takes as input 
1) dict: the dictionary of cities and their coordinates 

The function calculates the euclidean distance between them and stores this distance in a 2x2 matrix called main_list'''
def cost(dict):
    main_list = []
    for coordinates in dict.values():
        x = coordinates[0]
        y = coordinates[1]
        list = []
        for c in dict.values():
            x1 = c[0]
            y1 = c[1]
            dist = sqrt((int(x)-int(x1))**2 + (int(y)-int(y1))**2)
            list.append((dist))
        main_list.append(list)
    #print('\nThe Euclidean distance between cities is:',main_list)
    return main_list

'''The heuristic() function takes as the following input:
1) dictionary: the dictionary containing city names and their coordinates
2) start_node: the node whose heuristic value we have to calculate
3) previous_path: the path of reaching from A to start_node
4) main_list: the 2x2 matrix having the euclidean distance between cities

The function calculates the heuristic value using the minimum span tree. 
It calculates the minimum distance it takes for going from start_node back to A through the nodes not in the previous_path
It returns the heuristic value'''
    
def heuristic(dictinary, start_node, previous_path, main_list):
    #heuristic_dict = {}
    not_visited = [item for item in dictionary if item not in previous_path]
    not_visited.append('A')
    visited = []
    edges = []
    sum = 0
    min_edge = [None,None,float('inf')]
    visited.append(start_node)
    not_visited.remove(start_node)
    while (len(not_visited))!=0:
        for i in visited:
            for j in not_visited:
                x = ord(i)-ord('A')
                y = ord(j)-ord('A')
                edges.append([i,j,main_list[x][y]])
        #print("The edges from which we select min value are:")    
        #print(edges)
        
        for e in range(0, len(edges)):
            dist = edges[e][2]
            if dist<min_edge[2] and edges[e][1] not in visited:
                min_edge = edges[e]
                
        #print("the minimum edge out of them is:")
        #print(min_edge)
        sum = sum + min_edge[2]
        vertex = min_edge[1]
        visited.append(vertex)
        not_visited.remove(vertex)
        min_edge = [None,None,float('inf')]
    #print("the heuristic value is:")
    #print(sum)
    return sum
    
            
                
'''The children() function takes as input
1) dictionary: the dictionary containing the cities and their coordinates
2) the current path traversed by the parent node

The function finds the children nodes of the parent_node, which are not in the path
It returns the child'''
        
def children(dictionary, path):
    child = []
    for i in dictionary:
        a = 0
        for j in path:
            if i == j:
                a = 1
                break
        if a==0:
            child.append(i)
    return child
        

'''A_star() function takes as input
1) dictionary: the dictionary containing the cities and their coordinates
2) main_list: the 2x2 matrix having the euclidean distance between cities

It starts from A and stores it's children in the priority_queue. 
The priority queue gives the path having the smallest value of f(n), it's children are then stored in the priority queue
and the whole process is repeated until we traverse all the cities '''

def A_star(dictionary, main_list):
    
    '''Define all the variables'''
    number_of_nodes = 0
    nodeA = Node('A', 0, 0, 0, [])
    path = nodeA.path_list('A', [])
    length = len(path)
    closed_set = []
    child_nodes = []
    open_set = PriorityQueue()
    open_set.put((1, nodeA))
    
    '''continue till we reach the goal state or until the priority_queue is empty'''
    while not open_set.empty():
        parent_node = open_set.get()
        number_of_nodes = number_of_nodes + 1
        
        '''This portion checks whether all the nodes are visited or not by comparing the dictionary of cities 
        with the path of the node removed from the priority queue. If all the cities are traversed, then it prints the best path, 
        the number of nodes traversed to reach the path and the cost and returns back to the main function'''
        node_list = []
        node_path = []
        
        node_path = parent_node[1].path_node
        node_path_original = node_path[:]
        for i in dictionary:
            node_list.append(i)
        node_path.sort()
        node_list.sort()
        if node_path == node_list:
            print("Best path is")
            print(node_path_original)
            print("Number of nodes visited are:", number_of_nodes)
            total_cost = parent_node[1].f
            print("the total cost is",total_cost)
            return
        
        '''If we have not reached the goal state, store the extracted node in closed_set and find it's children '''
        closed_set.append(parent_node)
        '''get the children of the node'''
        child_nodes = children(dictionary, node_path_original)
        
        '''for every child, generate it's g,h and f and put the node in the priority_queue'''
        for node in child_nodes:
            #find the euclidean distance between the current extracted node and its child
            x = (ord(parent_node[1].n)) - ord('A')
            y = (ord(node)) - ord('A')
            euc_distance = main_list[x][y]
            #the total distance of the child node from the child node
            g_child = euc_distance + parent_node[1].g
            #heuristic value
            #h_child = 0
            h_child = heuristic(dictionary, node, node_path_original, main_list)
            f_child = g_child + h_child
            #make an object of the class Node
            node_path_temporary = node_path_original[:]
            node_path_temporary.append(node)
            node_child = Node(node, g_child, h_child, f_child, node_path_temporary)
            open_set.put((f_child, node_child))
        
            
                        
if __name__=='__main__':
    
    "Enter the entire path of the file where the question is placed"
    print("Enter the entire path of the file eg(E:\\waterloo_documents\\AI\\TSPproblem\\randTSP\\9\\instance_1.txt)")
    file_name = input()
    dictionary = readFile(file_name)
    main_list = cost(dictionary)
    A_star(dictionary, main_list)
    
    

Enter the entire path of the file eg(E:\waterloo_documents\AI\TSPproblem\randTSP\9\instance_1.txt)
E:\waterloo_documents\AI\TSPproblem\randTSP\16\instance_1.txt
The number of cities are : 16

The cities and their coordinates are: {'A': ['14', '82'], 'B': ['90', '67'], 'C': ['21', '69'], 'D': ['42', '29'], 'E': ['69', '28'], 'F': ['62', '19'], 'G': ['56', '1'], 'H': ['19', '13'], 'I': ['53', '9'], 'J': ['82', '11'], 'K': ['29', '44'], 'L': ['42', '77'], 'M': ['6', '44'], 'N': ['54', '60'], 'O': ['46', '40'], 'P': ['87', '99']}
Best path is
['A', 'C', 'K', 'M', 'H', 'I', 'G', 'J', 'E', 'F', 'D', 'O', 'N', 'B', 'P', 'L']
Number of nodes visited are: 23108
the total cost is 404.0205022000922
